In [1]:
from ETGEMs_function_PMI import *
import pandas as pd
import cobra
import ast
from cobra.io import write_sbml_model
from numpy import *
import copy

In [2]:
#Get Reaction G0 from local file_original
reaction_g0_file_original0 = './reaction_g0_strain0.txt'
reaction_g0_file_original1 = './reaction_g0_strain1.txt'
#Get Metabolite concentration from local file
metabolites_lnC_file_original0 = './metabolites_lnC_strain0.txt'
metabolites_lnC_file_original1 = './metabolites_lnC_strain1.txt'
#Get Model from local file
model_file_original0 = './iML1515.xml'
model_file_original1 = './iMM904.xml'
#Get reaction kcat data from ECMpy
reaction_kcat_MW_file_original0 = './ID_kcat_MW_file_strain0.csv'
reaction_kcat_MW_file_original1 = './ID_kcat_MW_file_strain1.csv'

In [3]:
## Convert to usable model formats
model0=Get_Concretemodel_Need_Data(reaction_g0_file_original0,metabolites_lnC_file_original0,model_file_original0,reaction_kcat_MW_file_original0)
model1=Get_Concretemodel_Need_Data(reaction_g0_file_original1,metabolites_lnC_file_original1,model_file_original1,reaction_kcat_MW_file_original1)
## knockout the btuB gene
model0['lb_list']['EX_cbl1_e ']=0
model0['lb_list']['EX_cbl1_e ']=0

In [4]:
# get the information of km, vmax and public metabolites
km = pd.read_csv('./km.csv')
vmax = pd.read_csv('./vmax.csv')
public_metabolism = pd.read_csv('./public_metabolism_test.csv')

'''
public_metabolism_name_list = []
public_metabolism_concentration_list = []
for i in public_metabolism['metabolism']:
    public_metabolism_name_list.append(i)
for j in public_metabolism['concentration']:
    public_metabolism_concentration_list.append(j)
public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
print(public_metabolism)
'''

"\npublic_metabolism_name_list = []\npublic_metabolism_concentration_list = []\nfor i in public_metabolism['metabolism']:\n    public_metabolism_name_list.append(i)\nfor j in public_metabolism['concentration']:\n    public_metabolism_concentration_list.append(j)\npublic_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))\nprint(public_metabolism)\n"

In [5]:
# definate the function to simulate metabolism and interaction
def time_space_state(model_list, biomass_list, growth_list, breed_list, parameter_list, public_metabolism, vmax, km, deta_t, deta_x, micro_distribute_threshold, length, D, o2_concentration):
    
    number_cell_side = length // deta_x
    number_cell_side = int(number_cell_side)
    public_metabolism_name_list = []
    public_metabolism_concentration_list = []
    for i in public_metabolism['metabolism']:
        public_metabolism_name_list.append(i)
    for j in public_metabolism['concentration']:
        public_metabolism_concentration_list.append(j)
    public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
    
    number_model = len(model_list)
    
    k_m = {}
    v_max = {}
    for i in range(number_model):
        for j in range(len(km['reactions_strain'+str(i)])):
            k_m[(i, km['reactions_strain'+str(i)][j])] = km['km_strain'+str(i)][j]
            v_max[(i, vmax['reactions_strain'+str(i)][j])] = vmax['vmax_strain'+str(i)][j]
    v_max_o2 = v_max[(0,'EX_glc__D_e_reverse')]
    k_m_o2 = k_m[(0,'EX_glc__D_e_reverse')]        
    
    number_public_metabolism = len(public_metabolism)
    distribute_micro_list = {}
    distribute_public_metabolism_list = {}
    distribute_lb_list = {}
    public_metabolism_r_list = []
    #set the initial distribution of straints
    for i in range(number_model):
        distribute_micro = np.random.randint(20, size=number_cell_side)
        distribute_micro_list.update({i: distribute_micro+1})
    print(distribute_micro_list)
    distribute_micro_list_initial = copy.deepcopy(distribute_micro_list)
    #set the initial distribution of metabolites
    for i in public_metabolism_name_list:
        distribute_public_metabolism = multiply(np.mat(np.ones(number_cell_side)), public_metabolism[i])
        distribute_public_metabolism_list.update({i: distribute_public_metabolism})
    #calculate the upperbounds of uptake reactions for public metabolites
    public_reaction_ub_list = {}
    public_reaction_list = {}
    for i in range(number_model):
        public_reaction_ub = {}
        public_reaction = []
        for rea in model_list[i]['reaction_list']:
            if 'EX_' in rea:
                for j in public_metabolism_name_list:
                    try:
                        model_list[i]['coef_matrix'][(j,rea)]
                    except:
                        pass
                    else:
                        ub = np.mat(np.ones(number_cell_side))
                        if model_list[i]['coef_matrix'][(j,rea)] > 0:
                            for m in range(number_cell_side):
                                ub[0,m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                        else:
                            try:
                                model_list[i]['ub_list'][rea]
                            except:
                                ub = ub * 1000
                            else:
                                ub = ub * model_list[i]['ub_list'][rea]
                        public_reaction_ub.update({rea: ub})
                        public_reaction.append(rea)
                        break          
            public_reaction_ub_list[i] = public_reaction_ub
            public_reaction_list[i] = public_reaction

            
    ct = 0
    
    distribute_micro_list_t = {ct: distribute_micro_list}
    distribute_public_metabolism_list_t = {ct: distribute_public_metabolism_list}
    distribute_lb_list_t = {ct: distribute_lb_list}
    r = micro_distribute_threshold + 1
    r_threshold = [r]*5
    slip_r = np.mean(r_threshold[-5:])
    number = {}
    various = {}
    for i in range(number_model):
        number[i] = [np.mean(distribute_micro_list[i])]
        various[i] = [np.std(distribute_micro_list[i])/np.mean(distribute_micro_list[i])]
    
            
    # iterative simulation by slip_r
    biomass = {}
    while slip_r >= micro_distribute_threshold:
        ct += 1
        print(ct)
        distribute_micro_list_t[ct] = copy.deepcopy(distribute_micro_list_t[ct-1])
        micro_decrease = {}
        micro_increase = {}
        
        #simulate the cell wandering
        #micro_decrease: the decrease number of strains
        #micro_increase: the increase number of strains
        #micro_decrease_cell: the decrease number of strains in the current grid
        #micro_increase_fcell: the increase number of strains in the forward grid
        #micro_increase_bcell: the increase number of strains in the backward grid
        for m in range(number_cell_side):
            micro_decrease_cell = {}
            micro_increase_cell = {}
            for i in range(number_model):
                micro_decrease_cell[i] = 0
                micro_increase_cell[i] = 0
            micro_decrease[m] = micro_decrease_cell
            micro_increase[m] = micro_increase_cell
            
            
        if ct > 0:
            met = 'glc__D_e'
            threshold = 0.3
            for m in range(number_cell_side):
                #calculate the number of strains in the internal grids
                if 0<m<number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = (distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * micro_decrease_cell
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_increase_bcell = micro_decrease_cell-micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                                    micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                                else:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                            elif distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the first grid
                elif m == 0:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the last grid
                elif m == number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_fcell = micro_increase_fcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
        
            #calculate the number of strains after wandering
            for m in range(number_cell_side):
                for i in range(number_model):
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] - micro_decrease[m][i]
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] + micro_increase[m][i]
                    distribute_micro_list_t[ct][i][m] = max(0, distribute_micro_list_t[ct][i][m])
                            
                            
        
        #simulate the substrate diffusion by Fick's second law
        for m in range(number_cell_side): 
            if 0<m<number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + ((distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x-(distribute_public_metabolism_list[met][0,m]-distribute_public_metabolism_list[met][0,m-1])/deta_x)/deta_x*D*deta_t
                    if distribute_public_metabolism_list[met][0,m] < 0:
                        print('Warning: the D is too high!')
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == 0:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m-1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
        
        #simulate the metabolism by ETMs
        biomass_value_list = {}
        number_model_range = []
        biomass_strain = {}
        for m in range(number_cell_side):
            biomass_strain[m] = [0, 0]
            if m%2 == 0:
                number_model_range.append([0,1])
            elif m%2 == 1:
                number_model_range.append([1,0])
            
            
            biomass_value_micro = {}
            for i in number_model_range[m]:
                if distribute_micro_list_t[ct][i][m] > 0:
                    public_metabolism_flux_list = {}
                    for j in public_metabolism_name_list:
                        public_metabolism_flux_list.update({j: 0})
                    for j in public_reaction_list[i]:
                        model_list[i]['ub_list'][j] = public_reaction_ub_list[i][j][0,m]
                    model_list[i]['ub_list']['EX_o2_e_reverse'] = min(v_max_o2*o2_concentration/(o2_concentration+k_m_o2), model_list[i]['ub_list']['EX_o2_e_reverse'])   
            
                    biomass_id = biomass_list[i]
                    E_total=parameter_list[i][0]
                    #set the carbon source to glucose
                    substrate_name='EX_glc__D_e_reverse'
                    substrate_value=parameter_list[i][1]
                    biomass_value=growth_list[i]
                    K_value=parameter_list[i][2]

                    #calculate the MDF values of metabolic networks
                    biomass_value_micro[i] = biomass_value
                    obj_name=biomass_list[i]
                    obj_target='maximize'
                    if i == 0:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation0(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation1(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    biomass_value=max_biomass_under_mdf*0.9
                        
                    #calculate the minimum value of the sum of the fluxes
                    if i == 0:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation0(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation1(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
            
                    #translating the results of ETMs into a usable form
                    model=model_list[i]['model']
                    reaction_kcat_MW=model_list[i]['reaction_kcat_MW']
                    reaction_g0=model_list[i]['reaction_g0']
                    coef_matrix=model_list[i]['coef_matrix']
                    metabolite_list=model_list[i]['metabolite_list']
                    use_result = Get_Results_Thermodynamics(model,Concretemodel,reaction_kcat_MW,reaction_g0,coef_matrix,metabolite_list)
            
                    if distribute_micro_list_t[ct][i][m] == 0:
                        biomass_strain[m][i] = 0
                    else:
                        biomass_strain[m][i] = use_result['flux'][biomass_list[i]]
                
                    #simulate the fluxes of the public metabolites
                    for rea in public_reaction_list[i]:
                        for met in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(met,rea)]  
                            except:
                                pass
                            else:
                                public_metabolism_flux_list[met] = public_metabolism_flux_list[met] + model_list[i]['coef_matrix'][(met,rea)]* use_result['flux'][rea]
                        
                    #simulate the distribution of the public metabolites
                    distribute_public_metabolism_ori = copy.deepcopy(distribute_public_metabolism_list)
                    distribute_public_metabolism_re = copy.deepcopy(distribute_public_metabolism_ori)
                    for met in public_metabolism_name_list:
                        distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] - public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m]
                        distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                        if distribute_public_metabolism_list[met][0,m] < 0:
                            print(met+'_1: ', distribute_public_metabolism_list[met][0,m])
                                
                    #simulate the multiplication and death rates of strains and the distribution of public metabolites after multiplication or death        
                    death_rate = 0
                    birth_rate = 1
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                bd_rate_met = (distribute_public_metabolism_list[met][0,m] - 0.1)/intracellular_c_dict[met]/distribute_micro_list_t[ct][i][m]
                                if bd_rate_met < 0:
                                    death_rate_lb = (public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] - distribute_public_metabolism_re[met][0,m] + 0.1)/(public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] + intracellular_c_dict[met] * distribute_micro_list_t[ct][i][m])
                                    death_rate = max(death_rate, death_rate_lb)
                                else:
                                    birth_rate = min(birth_rate, bd_rate_met)
                    death_rate = min(death_rate, 1)
                    birth_rate = min(birth_rate,1)
                    if death_rate > 0:
                        birth_rate = 0
                        for met in public_metabolism_name_list:
                            distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + public_metabolism_flux_list[met] * deta_t * math.ceil(distribute_micro_list_t[ct][i][m] * death_rate)
                            distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                    
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_ori[met][0,m] - intracellular_c_dict[met]*math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i]) + intracellular_c_dict[met]*math.ceil(death_rate*distribute_micro_list_t[ct][i][m])
                                if distribute_public_metabolism_list[met][0,m] < 0:
                                    print(met+'_2: ', distribute_public_metabolism_list[met][0,m])
                                    distribute_public_metabolism_list[met][0,m] = 0
                        
                    #simulate the distribution of strains after multiplication or death
                    distribute_micro_increase = math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i])
                    distribute_micro_decrease = math.ceil(distribute_micro_list_t[ct][i][m]*death_rate)
                    distribute_micro_list_t[ct][i][m] = max(0,distribute_micro_list_t[ct][i][m] + distribute_micro_increase - distribute_micro_decrease)
            
            #calculate the upperbounds of nutrient uptake rates after substrate diffusion, cell wandering, metabolism, multiplication and death
            biomass_value_list[m] = biomass_value_micro
            
            for i in range(number_model):
                for rea in model_list[i]['reaction_list']:
                    if 'EX_' in rea:
                        for j in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(j,rea)]
                            except:
                                pass
                            else:
                                if model_list[i]['coef_matrix'][(j,rea)] > 0:
                                    public_reaction_ub_list[i][rea][0, m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                   
        
        if ct == 1:
            biomass_strain_initial = copy.deepcopy(biomass_strain)
        
        # calculate the mean number and uniformity of distribution of strains at this iteratio
        for i in range(number_model):
            strain_number = np.mean(distribute_micro_list_t[ct][i])
            strain_various = np.std(distribute_micro_list_t[ct][i])
            number[i].append(strain_number)
            various[i].append(strain_various)
            print('strain_number: ', i, strain_number)
            print('strain_various: ', i, strain_various)
        
        # calculate the slip_r at this iteration
        if ct > 1:
            r = 0
            for i in range(number_model):
                for m in range(number_cell_side):
                    if distribute_micro_list_t[ct-1][i][m] > 0:
                        r = r + abs((distribute_micro_list_t[ct][i][m]-distribute_micro_list_t[ct-1][i][m])/(distribute_micro_list_t[ct-1][i][m])/deta_t)
                    else:
                        r = r + abs(distribute_micro_list_t[ct][i][m]/deta_t)
       
        r_threshold.append(r)
        slip_r = np.mean(r_threshold[-1:])
        fd_r_threshold = r_threshold[1:]
        print('fd_r_threshold: ', fd_r_threshold)
        print('slip_r: ', slip_r)
        
        biomass_x = {}
        biomass_x_initial = {}
        for i in range(number_model):
            biomass_x[i] = [1]*number_cell_side
            if ct == 1:
                biomass_x_initial[i] = np.ones(number_cell_side)
            for m in range(number_cell_side):
                biomass_x[i][m] = distribute_micro_list_t[ct][i][m]*biomass_strain[m][i]
                if ct == 1:
                    biomass_x_initial[i][m] = distribute_micro_list_initial[i][m]*biomass_strain_initial[m][i]
        biomass[ct] = biomass_x
        
        if ct == 1:
            print('_1: ', distribute_micro_list_initial)
            print('_2: ', biomass_strain_initial)
            print('_3: ', biomass_x_initial)
            
    return(biomass_x_initial, biomass, number, various)

In [6]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.29*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 7,  2,  6, 15, 14, 17,  7, 16, 20,  3, 17,  8,  5, 14,  2, 12,  4,
        7, 11, 16, 19,  4,  7, 14, 15, 10,  1,  4,  3,  2]), 1: array([13, 20, 11,  9, 13, 16,  6,  1,  4, 14,  8, 18,  2, 11,  6,  1, 19,
        8,  3, 10, 16, 15,  5, 10,  8,  9, 16, 20,  6, 10])}
1
strain_number:  0 10.866666666666667
strain_various:  0 6.989198014714484
strain_number:  1 10.266666666666667
strain_various:  1 5.507167047483569
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 7,  2,  6, 15, 14, 17,  7, 16, 20,  3, 17,  8,  5, 14,  2, 12,  4,
        7, 11, 16, 19,  4,  7, 14, 15, 10,  1,  4,  3,  2]), 1: array([13, 20, 11,  9, 13, 16,  6,  1,  4, 14,  8, 18,  2, 11,  6,  1, 19,
        8,  3, 10, 16, 15,  5, 10,  8,  9, 16, 20,  6, 10])}
_2:  {0: [0.6345187588838144, 0.2857391744693639], 1: [0.6345187588838144, 0.2857391744693639], 2: [0.6345187588838144, 0.2857391744693639], 3: [0.6345187588838144, 0.2857391744693639], 4: [0.6345187588838144, 0.2857391744693639], 

glc__D_e_1:  -3.216234201480372
glc__D_e_1:  -2.8643851624867267
glc__D_e_1:  -5.197010344839679
glc__D_e_1:  -1.7200838728937455
glc__D_e_1:  -7.47418707566597
glc__D_e_1:  -8.165580049415668
glc__D_e_1:  -7.435559235678838
glc__D_e_1:  -10.764887325142771
glc__D_e_1:  -4.252339144462006
strain_number:  0 38.56666666666667
strain_various:  0 19.580063556814338
strain_number:  1 0.6
strain_various:  1 1.1718930554164628
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 745.5081968470128, 350.9491867781341, 395.0662878787879, 415.03414085914085, 385.7281202102934, 406.238164583095, 386.3756000841915, 435.5764552955343, 395.8638903134232, 359.94569425205657, 362.3801209884342, 460.03000401656794, 437.2690410400903, 265.4977824749923]
slip_r:  265.4977824749923
16
glc__D_e_1:  -26.46621853883864
glc__D_e_1:  -2.1689483462245995
glc__D_e_1:  -0.24577970192899667
glc__D_e_1:  -25.24515399410186
glc__D_e_1:  -15.841770512461327
glc__D_e_1:  -11.958632702395823
glc__D_e_1:  -14.94169740806708
glc__D

glc__D_e_1:  -0.017539439738562645
glc__D_e_1:  -0.7526750912771598
glc__D_e_1:  -2.4619076390039276
glc__D_e_1:  -2.543917940023282
glc__D_e_1:  -7.225102360495059
glc__D_e_1:  -9.551537751701863
strain_number:  0 9.1
strain_various:  0 21.384340064636085
strain_number:  1 0.03333333333333333
strain_various:  1 0.17950549357115014
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 745.5081968470128, 350.9491867781341, 395.0662878787879, 415.03414085914085, 385.7281202102934, 406.238164583095, 386.3756000841915, 435.5764552955343, 395.8638903134232, 359.94569425205657, 362.3801209884342, 460.03000401656794, 437.2690410400903, 265.4977824749923, 206.26593876103922, 269.63124802710166, 218.9175211931764, 202.6870514522396, 208.49753053963585, 227.40800865800867]
slip_r:  227.40800865800867
22
glc__D_e_1:  -0.3629121664578763
glc__D_e_1:  -1.9994939973969243
glc__D_e_1:  -0.5965372862658646
glc__D_e_1:  -1.2059262590417315
glc__D_e_1:  -1.329013476052313
glc__D_e_1:  -0.6900339628369074
glc__D_e_

In [7]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.29*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 3, 12,  2,  2,  4,  7, 17, 13,  3,  4, 16,  3, 12,  7, 14, 18, 10,
       13,  1,  9, 12, 17,  7,  3, 15,  2, 10,  8, 18,  6]), 1: array([ 7,  9,  6, 10, 19,  8, 15,  4, 20,  1,  5,  9,  6, 20, 11,  2,  2,
        7,  4,  4, 15,  8, 15,  8, 13, 17, 13,  7,  8,  3])}
1
strain_number:  0 10.266666666666667
strain_various:  0 6.557099629832961
strain_number:  1 9.2
strain_various:  1 5.393823628311677
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 3, 12,  2,  2,  4,  7, 17, 13,  3,  4, 16,  3, 12,  7, 14, 18, 10,
       13,  1,  9, 12, 17,  7,  3, 15,  2, 10,  8, 18,  6]), 1: array([ 7,  9,  6, 10, 19,  8, 15,  4, 20,  1,  5,  9,  6, 20, 11,  2,  2,
        7,  4,  4, 15,  8, 15,  8, 13, 17, 13,  7,  8,  3])}
_2:  {0: [0.6345187588838144, 0.2857391744693639], 1: [0.6345187588838144, 0.2857391744693639], 2: [0.6345187588838144, 0.2857391744693639], 3: [0.6345187588838144, 0.2857391744693639], 4: [0.6345187588838144, 0.2857391744693639], 5: [0.634518758

glc__D_e_1:  -1.059149990530905
glc__D_e_1:  -12.47693604495162
glc__D_e_1:  -21.65596881420997
glc__D_e_1:  -23.053513255454998
glc__D_e_1:  -20.808275320517204
glc__D_e_1:  -19.72512201166585
glc__D_e_1:  -21.47994421366937
glc__D_e_1:  -33.7379201015026
glc__D_e_1:  -21.710568847399834
glc__D_e_1:  -15.067516225718718
glc__D_e_1:  -19.046338031008744
glc__D_e_1:  -25.64730316508808
glc__D_e_1:  -18.97494874116648
glc__D_e_1:  -26.1046720364605
glc__D_e_1:  -33.84043070979001
glc__D_e_1:  -24.451692185523047
glc__D_e_1:  -26.674247105163698
glc__D_e_1:  -15.277369053570512
glc__D_e_1:  -11.949266760205767
glc__D_e_1:  -34.41610586498189
glc__D_e_1:  -18.534436986084756
glc__D_e_1:  -0.47644649163629643
glc__D_e_1:  -13.15722467270573
strain_number:  0 31.633333333333333
strain_various:  0 9.819990948174148
strain_number:  1 0.16666666666666666
strain_various:  1 0.37267799624996495
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 719.2815412142038, 388.1678321678321, 354.5805137844612, 400

glc__D_e_1:  -19.613329106559497
glc__D_e_1:  -14.6757973274892
glc__D_e_1:  -5.975669969605175
glc__D_e_1:  -3.9837919346588757
glc__D_e_1:  -2.7859360499637327
glc__D_e_1:  -0.4456792746174738
glc__D_e_1:  -0.5853701277109438
glc__D_e_1:  -0.908063098652135
glc__D_e_1:  -0.9161558904602286
glc__D_e_1:  -0.7356379770414045
glc__D_e_1:  -0.9576627090235382
glc__D_e_1:  -0.36217294144731416
glc__D_e_1:  -0.21865330979626507
glc__D_e_1:  -0.7029235716622213
glc__D_e_1:  -0.646811277423877
glc__D_e_1:  -0.9532935465888132
glc__D_e_1:  -0.4370406307740663
strain_number:  0 8.2
strain_various:  0 16.023732399163436
strain_number:  1 0.03333333333333333
strain_various:  1 0.1795054935711501
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 719.2815412142038, 388.1678321678321, 354.5805137844612, 400.35677480414324, 381.7108972898448, 426.62852162852164, 374.8796839529025, 377.579930508054, 382.93949498318136, 378.40686457073457, 305.2050278685349, 357.0689232144921, 331.91898214879336, 239.51837510

strain_number:  0 0.0
strain_various:  0 0.0
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 719.2815412142038, 388.1678321678321, 354.5805137844612, 400.35677480414324, 381.7108972898448, 426.62852162852164, 374.8796839529025, 377.579930508054, 382.93949498318136, 378.40686457073457, 305.2050278685349, 357.0689232144921, 331.91898214879336, 239.51837510410826, 244.83628025601183, 211.44641036074904, 195.26611340327116, 182.19009956425634, 180.34317830190497, 212.10868983957218, 213.55341288332409, 247.5519393111936, 160.59196848670533, 67.2381588999236, 37.96153846153846, 52.714285714285715, 61.964285714285715, 45.0, 22.5, 15.0, 0.0]
slip_r:  0.0
The initial distribution of microorganism is:  {}
The change process of microorganism distribution is:  {1: {0: [1.9035562766514431, 8.883262624373401, 1.2690375177676287, 1.2690375177676287, 2.5380750355352575, 5.076150071070515, 12.690375177676287, 9.517781383257216, 1.9035562766514431, 2.538075035535

In [8]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([18,  4, 10,  3, 13, 20,  3, 19, 19, 19, 19, 13, 20,  1, 11, 13,  4,
       14, 12,  1,  6, 16,  3, 20, 11, 11, 14,  6, 10, 17]), 1: array([10, 10, 17, 20,  3,  9,  3,  3, 12, 12,  7, 20, 20,  1,  7, 19, 19,
        3, 16, 14,  5, 17, 14,  7,  8, 12, 14, 16, 13, 13])}
1
strain_number:  0 13.566666666666666
strain_various:  0 7.486357963359457
strain_number:  1 11.466666666666667
strain_various:  1 5.713920623257632
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([18,  4, 10,  3, 13, 20,  3, 19, 19, 19, 19, 13, 20,  1, 11, 13,  4,
       14, 12,  1,  6, 16,  3, 20, 11, 11, 14,  6, 10, 17]), 1: array([10, 10, 17, 20,  3,  9,  3,  3, 12, 12,  7, 20, 20,  1,  7, 19, 19,
        3, 16, 14,  5, 17, 14,  7,  8, 12, 14, 16, 13, 13])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.28573917446936

glc__D_e_1:  -22.470508574371543
glc__D_e_1:  -1.0026559892772777
glc__D_e_1:  -36.107698860808725
glc__D_e_1:  -25.320193739215092
glc__D_e_1:  -0.14297368142707612
glc__D_e_1:  -0.0953741345105894
glc__D_e_1:  -13.359513045798845
glc__D_e_1:  -38.33695768496202
glc__D_e_1:  -0.14529126643094514
glc__D_e_1:  -21.697942369234273
glc__D_e_1:  -10.538248554116045
glc__D_e_1:  -19.0701179727863
glc__D_e_1:  -14.130017816708554
glc__D_e_1:  -19.608234960540248
glc__D_e_1:  -10.230278926376833
glc__D_e_1:  -13.051289103470832
glc__D_e_1:  -8.606156158206678
glc__D_e_1:  -0.15274993385547475
glc__D_e_1:  -21.382947134314076
glc__D_e_1:  -22.589722706827494
glc__D_e_1:  -1.1218701217332292
glc__D_e_1:  -16.853857030476554
glc__D_e_1:  -3.9786218319874465
glc__D_e_1:  -2.092277889325632
glc__D_e_1:  -6.562531678271604
glc__D_e_1:  -3.641516456646083
glc__D_e_1:  -5.522199296982205
glc__D_e_1:  -2.115508642189771
glc__D_e_1:  -19.906984811552334
glc__D_e_1:  -18.52338630923972
glc__D_e_1:  -1.4

glc__D_e_1:  -2.339708990786074
glc__D_e_1:  -3.1098137716771963
glc__D_e_1:  -2.962351278280335
glc__D_e_1:  -3.5302553657210165
glc__D_e_1:  -1.6780913614834203
glc__D_e_1:  -1.6125857616379133
glc__D_e_1:  -1.9551164828627323
glc__D_e_1:  -0.8224809898218008
glc__D_e_1:  -0.789118545520307
glc__D_e_1:  -0.3253481583480965
glc__D_e_1:  -0.04526924309379199
glc__D_e_1:  -0.3545874751929764
glc__D_e_1:  -0.6010910684940345
glc__D_e_1:  -0.946063693686118
glc__D_e_1:  -1.5475987385445689
glc__D_e_1:  -2.559977865623632
glc__D_e_1:  -2.967451113218559
glc__D_e_1:  -1.4738500558491725
glc__D_e_1:  -6.466652493235309
glc__D_e_1:  -6.537918976693361
glc__D_e_1:  -2.4427768392018967
glc__D_e_1:  -4.731862676949369
glc__D_e_1:  -1.1578139731776624
glc__D_e_1:  -1.360563346518572
glc__D_e_1:  -1.7529211407438363
glc__D_e_1:  -1.1735729103497599
glc__D_e_1:  -1.7801478668750916
glc__D_e_1:  -2.0209036600506263
glc__D_e_1:  -1.3025808676850366
glc__D_e_1:  -1.9707317673264781
strain_number:  0 3

In [9]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([10,  3,  7, 15,  7, 10, 18, 10, 18,  9, 20,  5, 12, 18, 16, 14, 12,
        1,  8,  4, 17, 18, 18,  1, 20, 20, 12,  5,  5, 18]), 1: array([15, 12,  8,  5,  6, 20, 16, 20, 12,  8, 17,  8, 11,  4,  2,  5,  4,
        7,  1, 20, 11, 12,  2, 18, 12,  1, 16, 18,  7,  9])}
1
strain_number:  0 13.7
strain_various:  0 7.184010022264723
strain_number:  1 10.233333333333333
strain_various:  1 5.914295299432459
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([10,  3,  7, 15,  7, 10, 18, 10, 18,  9, 20,  5, 12, 18, 16, 14, 12,
        1,  8,  4, 17, 18, 18,  1, 20, 20, 12,  5,  5, 18]), 1: array([15, 12,  8,  5,  6, 20, 16, 20, 12,  8, 17,  8, 11,  4,  2,  5,  4,
        7,  1, 20, 11, 12,  2, 18, 12,  1, 16, 18,  7,  9])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.498

glc__D_e_1:  -10.662395769133942
glc__D_e_1:  -19.95245926099554
glc__D_e_1:  -0.004953388031893269
glc__D_e_1:  -18.025910701633123
glc__D_e_1:  -23.598982295082802
glc__D_e_1:  -24.01496492248119
glc__D_e_1:  -33.15458252101008
glc__D_e_1:  -25.59198215442106
glc__D_e_1:  -16.42190384134569
glc__D_e_1:  -6.468678488057474
glc__D_e_1:  -24.460986171864086
glc__D_e_1:  -14.85220606536664
glc__D_e_1:  -17.780180022678397
glc__D_e_1:  -14.382460230920687
glc__D_e_1:  -18.409658801294615
glc__D_e_1:  -15.67203531944466
glc__D_e_1:  -19.512059141499243
glc__D_e_1:  -1.084899980666215
glc__D_e_1:  -18.43649422747827
glc__D_e_1:  -23.75443701379375
glc__D_e_1:  -0.7662377165688667
glc__D_e_1:  -1.1054055240067222
glc__D_e_1:  -9.686424026119406
glc__D_e_1:  -42.971636839163864
glc__D_e_1:  -2.893626477970976
glc__D_e_1:  -1.0532168499291
glc__D_e_1:  -12.756315624825497
strain_number:  0 40.53333333333333
strain_various:  0 21.105660114976004
strain_number:  1 1.1
strain_various:  1 1.325393

glc__D_e_1:  -0.9045979399709521
glc__D_e_1:  -0.5037604151278416
glc__D_e_1:  -1.356473101603822
glc__D_e_1:  -1.1193640392607485
glc__D_e_1:  -1.6834992180910968
glc__D_e_1:  -1.3826868179350398
glc__D_e_1:  -0.23696533354444949
strain_number:  0 4.433333333333334
strain_various:  0 2.512413624828169
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 995.4688727165508, 377.19375361480627, 400.93645784040524, 450.76886133579694, 467.6444009342366, 382.0206727352096, 410.60974590009346, 485.82233048600455, 509.58933427051784, 387.6969540607406, 306.32443595341095, 362.3810193990355, 311.47839175951833, 353.2761171832767, 222.1616479719842, 207.53495633170556, 211.47730455818694, 207.63571844821846]
slip_r:  207.63571844821846
20
glc__D_e_1:  -2.5611121934921153
glc__D_e_1:  -3.812038891942214
glc__D_e_1:  -4.059542165352432
glc__D_e_1:  -1.7100983824579643
glc__D_e_1:  -1.4686511162543443
glc__D_e_1:  -0.7776474798078321
glc__D_e_1:  -0.764789334989

In [10]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 7, 16, 18,  4,  1, 16, 19,  1, 11, 14,  1, 15,  7,  6, 11, 19,  7,
        4,  1,  1,  7, 14, 12,  5,  1, 15, 19, 20, 17, 17]), 1: array([13,  7, 10,  7,  1,  5,  2,  1,  1, 13,  1,  5, 19, 12,  6, 15,  3,
        7,  4,  8, 19,  1,  8, 16, 15,  7, 15,  7,  7,  8])}
1
strain_number:  0 11.866666666666667
strain_various:  0 7.830424992013878
strain_number:  1 8.1
strain_various:  1 5.3935146240647205
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 7, 16, 18,  4,  1, 16, 19,  1, 11, 14,  1, 15,  7,  6, 11, 19,  7,
        4,  1,  1,  7, 14, 12,  5,  1, 15, 19, 20, 17, 17]), 1: array([13,  7, 10,  7,  1,  5,  2,  1,  1, 13,  1,  5, 19, 12,  6, 15,  3,
        7,  4,  8, 19,  1,  8, 16, 15,  7, 15,  7,  7,  8])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.498

glc__D_e_1:  -2.274086330812789
glc__D_e_1:  -2.7886288009098337
glc__D_e_1:  -13.01544306533556
glc__D_e_1:  -19.718413704361954
glc__D_e_1:  -9.034362244392575
glc__D_e_1:  -12.77583733213584
glc__D_e_1:  -13.12432999423499
glc__D_e_1:  -7.913226015935201
glc__D_e_1:  -4.5260956078747725
glc__D_e_1:  -8.726247579592954
strain_number:  0 46.86666666666667
strain_various:  0 21.448750287345156
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 1092.1457424154794, 327.9427020348073, 295.1827464202464, 324.58726187870934, 431.9420929947246, 422.7056832056833, 418.0653463041622, 432.07901275059595, 356.94111522171295, 427.8811783605618, 394.4979724252174, 371.9461521959778, 235.1536107948887, 174.09761546267973]
slip_r:  174.09761546267973
16
glc__D_e_1:  -30.20721636407985
glc__D_e_1:  -28.466546017820594
glc__D_e_1:  -16.350692692775095
glc__D_e_1:  -33.119725805583855
glc__D_e_1:  -34.47389703045304
glc__D_e_1:  -23.322466079129477
glc__D_e_1:  -18.

glc__D_e_1:  -9.640922714775483
glc__D_e_1:  -15.261493889839029
glc__D_e_1:  -1.4621339960910738
glc__D_e_1:  -3.6865692510890002
strain_number:  0 5.533333333333333
strain_various:  0 5.51200709562517
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 1092.1457424154794, 327.9427020348073, 295.1827464202464, 324.58726187870934, 431.9420929947246, 422.7056832056833, 418.0653463041622, 432.07901275059595, 356.94111522171295, 427.8811783605618, 394.4979724252174, 371.9461521959778, 235.1536107948887, 174.09761546267973, 182.580068114106, 177.1625422304948, 171.1006492307255, 197.2357254781576, 207.37624727423113, 164.2187192231429]
slip_r:  164.2187192231429
22
glc__D_e_1:  -1.0228345877735694
glc__D_e_1:  -0.32453324613452206
glc__D_e_1:  -0.15749954591808724
glc__D_e_1:  -0.24967278089281464
glc__D_e_1:  -0.8078477410859384
glc__D_e_1:  -0.7782261443299712
glc__D_e_1:  -1.1423983071771833
glc__D_e_1:  -0.6525357081119332
glc__D_e_1:  -0.64547644429

In [11]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 3,  1, 14,  7, 18,  5, 15,  5,  6, 17,  7,  1, 17, 15, 17,  7, 13,
        6,  1,  6,  2,  2, 17, 18, 10, 16,  7,  7, 11,  6]), 1: array([ 4,  8, 14,  7, 20, 11, 10,  9, 11,  2,  4, 14,  2, 13, 19, 19,  1,
       10,  9, 11, 15,  7, 10, 11, 14,  2, 19,  9, 19, 10])}
1
strain_number:  0 10.766666666666667
strain_various:  0 6.8686890225783905
strain_number:  1 10.466666666666667
strain_various:  1 5.395883204402738
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 3,  1, 14,  7, 18,  5, 15,  5,  6, 17,  7,  1, 17, 15, 17,  7, 13,
        6,  1,  6,  2,  2, 17, 18, 10, 16,  7,  7, 11,  6]), 1: array([ 4,  8, 14,  7, 20, 11, 10,  9, 11,  2,  4, 14,  2, 13, 19, 19,  1,
       10,  9, 11, 15,  7, 10, 11, 14,  2, 19,  9, 19, 10])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693

glc__D_e_1:  -0.8423489822354355
glc__D_e_1:  -20.23069985816212
glc__D_e_1:  -25.46626530861783
glc__D_e_1:  -13.049035348888994
glc__D_e_1:  -28.44737355707713
glc__D_e_1:  -0.2294600750278768
glc__D_e_1:  -13.374866962818844
glc__D_e_1:  -24.07373483703754
glc__D_e_1:  -24.49724775406697
glc__D_e_1:  -27.15047728105784
glc__D_e_1:  -16.420649830149138
glc__D_e_1:  -15.676959644452129
glc__D_e_1:  -13.565933933716739
glc__D_e_1:  -18.120728988366757
glc__D_e_1:  -17.30419713961076
glc__D_e_1:  -11.916094895753556
glc__D_e_1:  -15.219486011210755
glc__D_e_1:  -13.269881317507899
glc__D_e_1:  -0.02168690137469631
glc__D_e_1:  -7.562011979226141
glc__D_e_1:  -15.630103733981338
glc__D_e_1:  -0.8357627967954433
glc__D_e_1:  -18.78302211725927
strain_number:  0 40.3
strain_various:  0 19.878212528628758
strain_number:  1 0.13333333333333333
strain_various:  1 0.42687494916218993
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 819.5720233713654, 374.46595115410906, 403.2566808191809, 394.734161

glc__D_e_1:  -0.18701820283027604
strain_number:  0 4.866666666666666
strain_various:  0 3.658171978218386
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 819.5720233713654, 374.46595115410906, 403.2566808191809, 394.7341616716617, 356.7533891547049, 449.6592417085838, 491.73568338848474, 492.72610367219875, 464.81060780617224, 470.6079561939526, 465.91148978957153, 433.6498217433112, 388.0984500967882, 400.49554190928905, 290.1593984643497, 207.28103119596358, 177.9036583479533, 172.71005350574075, 184.21604242702247, 180.9597608274079]
slip_r:  180.9597608274079
22
glc__D_e_1:  -6.043590015204732
glc__D_e_1:  -5.04722888904045
glc__D_e_1:  -3.6740081805477622
glc__D_e_1:  -1.3376886945665991
glc__D_e_1:  -1.4035266098724262
glc__D_e_1:  -1.219094476475957
glc__D_e_1:  -0.6726585744049121
glc__D_e_1:  -1.0288895243432465
glc__D_e_1:  -0.2518449776952153
glc__D_e_1:  -0.4015442240294925
glc__D_e_1:  -1.2641002365359506
glc__D_e_1:  -1.17160473270

In [12]:
#25
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 3,  7, 17,  6,  2,  9,  3,  9, 16, 16, 19,  9, 16,  1, 12,  8, 20,
        4,  5,  9,  7, 10, 13, 20, 12,  1, 11,  5, 11, 12]), 1: array([20,  1, 19,  9, 18, 17, 15, 11,  5,  2,  7, 15, 16,  6, 13,  4,  1,
       15,  9, 17, 10, 16,  9, 20,  1,  4, 18, 17, 20,  1])}
1
strain_number:  0 11.333333333333334
strain_various:  0 6.659996663329161
strain_number:  1 11.2
strain_various:  1 6.539113089708726
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 3,  7, 17,  6,  2,  9,  3,  9, 16, 16, 19,  9, 16,  1, 12,  8, 20,
        4,  5,  9,  7, 10, 13, 20, 12,  1, 11,  5, 11, 12]), 1: array([20,  1, 19,  9, 18, 17, 15, 11,  5,  2,  7, 15, 16,  6, 13,  4,  1,
       15,  9, 17, 10, 16,  9, 20,  1,  4, 18, 17, 20,  1])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.498

glc__D_e_1:  -1.072783371463089
glc__D_e_1:  -17.862725460391804
glc__D_e_1:  -16.96934032871887
glc__D_e_1:  -0.06252788001646437
glc__D_e_1:  -0.7209862174543338
glc__D_e_1:  -19.708938962179403
glc__D_e_1:  -26.393026008453685
glc__D_e_1:  -0.15206166269972687
glc__D_e_1:  -9.773773589073649
glc__D_e_1:  -21.141974658094746
glc__D_e_1:  -14.481258986035634
glc__D_e_1:  -24.59502499740203
glc__D_e_1:  -20.018841770689445
glc__D_e_1:  -24.800291865723363
glc__D_e_1:  -0.3362858340960253
glc__D_e_1:  -30.251519533564093
glc__D_e_1:  -16.165454261568385
glc__D_e_1:  -0.11031447656408488
glc__D_e_1:  -31.983222387542224
glc__D_e_1:  -41.893158828116086
glc__D_e_1:  -1.9981470821887939
glc__D_e_1:  -0.37842776411237977
glc__D_e_1:  -14.162913387531253
glc__D_e_1:  -29.157918409987133
glc__D_e_1:  -0.7570063126722815
glc__D_e_1:  -11.85779043818673
glc__D_e_1:  -28.466610031789052
glc__D_e_1:  -1.5860446466048224
glc__D_e_1:  -19.541010300085453
glc__D_e_1:  -19.165386956596738
glc__D_e_1:

glc__D_e_1:  -7.743685903129559
glc__D_e_1:  -1.8129016085799212
glc__D_e_1:  -5.139230158826877
glc__D_e_1:  -4.237090174859096
glc__D_e_1:  -1.9715384331672816
glc__D_e_1:  -3.475854128340522
glc__D_e_1:  -2.6685810604546396
glc__D_e_1:  -1.3843345039740274
glc__D_e_1:  -1.1897053890221059
glc__D_e_1:  -1.5511046749138333
glc__D_e_1:  -0.0259911448601593
glc__D_e_1:  -0.4832166423659042
glc__D_e_1:  -1.6576851424745431
glc__D_e_1:  -1.9003102085235963
glc__D_e_1:  -2.978601762895479
glc__D_e_1:  -2.0578671916336866
glc__D_e_1:  -0.9113969095750154
glc__D_e_1:  -3.0733307140478017
glc__D_e_1:  -2.069498134132677
glc__D_e_1:  -0.7499541748770466
glc__D_e_1:  -0.7593351678287323
glc__D_e_1:  -1.7396839055348063
glc__D_e_1:  -2.5445502567566773
glc__D_e_1:  -0.7128201221197065
glc__D_e_1:  -2.0460953661774064
glc__D_e_1:  -1.719235170548644
glc__D_e_1:  -2.30726241589683
glc__D_e_1:  -1.8275842591291047
glc__D_e_1:  -2.2906634049768986
glc__D_e_1:  -2.0521024457588344
strain_number:  0 3

In [13]:
#25
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 2,  1, 20,  2, 13,  1,  8,  8,  6, 18, 10, 14, 18,  7, 12, 20,  1,
       13,  6,  3,  3,  2,  9,  3,  1, 20,  1,  5, 12, 12]), 1: array([ 9, 17, 12,  1,  4,  2, 19, 15, 11,  5, 20,  2, 14,  5, 15, 14, 18,
        6, 18, 15,  6,  3,  9, 19, 18, 12, 17,  9,  9,  9])}
1
strain_number:  0 9.666666666666666
strain_various:  0 7.634716031974529
strain_number:  1 11.1
strain_various:  1 5.810048766289889
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1]
slip_r:  1.1
_1:  {0: array([ 2,  1, 20,  2, 13,  1,  8,  8,  6, 18, 10, 14, 18,  7, 12, 20,  1,
       13,  6,  3,  3,  2,  9,  3,  1, 20,  1,  5, 12, 12]), 1: array([ 9, 17, 12,  1,  4,  2, 19, 15, 11,  5, 20,  2, 14,  5, 15, 14, 18,
        6, 18, 15,  6,  3,  9, 19, 18, 12, 17,  9,  9,  9])}
_2:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.4988

glc__D_e_1:  -5.27494874482635
strain_number:  0 36.1
strain_various:  0 16.131439282758787
strain_number:  1 0.8666666666666667
strain_various:  1 0.9568466729604882
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 1029.2837491455914, 379.47687458310537, 342.40930998825746, 446.93374169689974, 508.4973887294557, 474.8822427572428, 417.2716133582621, 484.23482407528456, 502.8715738109839, 460.6794879882558, 403.2014747995311, 370.1220950671669, 390.5042026470485, 396.86926272410864]
slip_r:  396.86926272410864
16
glc__D_e_1:  -5.88083160914476
glc__D_e_1:  -17.993670411196057
glc__D_e_1:  -16.08868482914653
glc__D_e_1:  -9.070216483605916
glc__D_e_1:  -7.088272545216755
glc__D_e_1:  -12.7259497983501
glc__D_e_1:  -2.0991796285620388
glc__D_e_1:  -10.499821921461118
glc__D_e_1:  -4.129515729695608
glc__D_e_1:  -9.122345178565059
glc__D_e_1:  -13.612340622311102
strain_number:  0 35.766666666666666
strain_various:  0 18.496275901440868
strain_number:  1 0.4
strain_various:  1 0.66332495807108


glc__D_e_1:  -0.14442535898288533
glc__D_e_1:  -2.8502226085156543
glc__D_e_1:  -3.543631522530526
glc__D_e_1:  -10.16067188032467
glc__D_e_1:  -17.510694025194578
glc__D_e_1:  -17.447351669928718
glc__D_e_1:  -29.21292120953623
glc__D_e_1:  -25.389515612881507
glc__D_e_1:  -20.152123648948063
glc__D_e_1:  -5.522648527901631
glc__D_e_1:  -2.9566942343530016
glc__D_e_1:  -1.9774673940573717
glc__D_e_1:  -0.4833928202425415
strain_number:  0 7.9
strain_various:  0 10.571502573743558
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 1029.2837491455914, 379.47687458310537, 342.40930998825746, 446.93374169689974, 508.4973887294557, 474.8822427572428, 417.2716133582621, 484.23482407528456, 502.8715738109839, 460.6794879882558, 403.2014747995311, 370.1220950671669, 390.5042026470485, 396.86926272410864, 309.21493191484933, 239.91566371413975, 250.22514162683305, 191.76859529618196, 189.14696549258085, 225.9997215499341, 212.71886582198826]
slip_r:  212.71